In [13]:
import ee
import pandas as pd
import requests
from io import StringIO

# Initialize the Earth Engine module.
# ee.Authenticate()
ee.Initialize()

# URL of the CSV file
url = 'https://raw.githubusercontent.com/orwell2024/uscrnlib/main/extract_slides/2024stations_days.csv'

# Read the CSV file from the URL
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# Limiting to first 10 locations for testing purposes
data = data.head(4)

# Function to create and export image for a given location
def create_and_export_image(row):
    station_name = row['Station']
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    
    point = ee.Geometry.Point([longitude, latitude])
    buffer = point.buffer(10000).bounds()  # 5000 meters buffer for 10 km x 10 km region
    
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(buffer) \
                    .filterDate('2023-05-01', '2023-6-01') \
                    .sort('CLOUDY_PIXEL_PERCENTAGE') \
                    .first()
    
    # Get the date of the image
    date = ee.Date(collection.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    
    # Select the relevant bands
    image = collection.select(['B4', 'B3', 'B2']).clip(buffer)
    
    # Export the image with smaller dimensions
    export_task = ee.batch.Export.image.toDrive(
        image=image,
        description=f"{station_name}_{date}",
        folder='GEE_Images',
        scale=10,
        region=buffer.getInfo()['coordinates'],
        fileFormat='GeoTIFF',  # Export as GeoTIFF
        maxPixels=1e8
    )
    
    export_task.start()

# Apply the function to each row in the dataframe
data.apply(create_and_export_image, axis=1)

print("Export tasks have been started.")


Export tasks have been started.
